In [1]:
import numpy as np
import pandas as pd
import glob
import os

In [2]:
dirs = glob.glob('input/dataset/train/*')
dirs

['input/dataset/train/7',
 'input/dataset/train/3',
 'input/dataset/train/14',
 'input/dataset/train/2',
 'input/dataset/train/0',
 'input/dataset/train/11',
 'input/dataset/train/4',
 'input/dataset/train/8',
 'input/dataset/train/5',
 'input/dataset/train/1',
 'input/dataset/train/9',
 'input/dataset/train/6',
 'input/dataset/train/13',
 'input/dataset/train/10',
 'input/dataset/train/12']

In [3]:
images = list()
labels = list()
for d in dirs:
    images += glob.glob(d + '/*')
    labels += [int(os.path.basename(d))] * len(glob.glob(d + '/*'))

In [4]:
images[:10], labels[:10]

(['input/dataset/train/7/n04602956_3862_0.jpg',
  'input/dataset/train/7/n04197391_3278_0.jpg',
  'input/dataset/train/7/n04197391_10112_0.jpg',
  'input/dataset/train/7/n04197391_11085_1.jpg',
  'input/dataset/train/7/n04602956_3082_1.jpg',
  'input/dataset/train/7/n04602956_2792_0.jpg',
  'input/dataset/train/7/n04197391_13180_0.jpg',
  'input/dataset/train/7/n03238879_2797_0.jpg',
  'input/dataset/train/7/n03238879_13857_0.jpg',
  'input/dataset/train/7/n04602956_3101_0.jpg'],
 [7, 7, 7, 7, 7, 7, 7, 7, 7, 7])

In [5]:
from sklearn.utils import shuffle
df = pd.DataFrame({'images': images, 'labels': labels})
df = shuffle(df)
df.head()

,images,labels
22424,input/dataset/train/4/n02780704_398_0.jpg,4
47337,input/dataset/train/6/n02667093_7166_0.jpg,6
36705,input/dataset/train/1/n02956883_8788_0.jpg,1
20162,input/dataset/train/4/n03132776_13829_0.jpg,4
30535,input/dataset/train/8/n03062015_3999_0.jpg,8


In [6]:
df.shape

(62258, 2)

In [7]:
train = df.iloc[:52258, :]
validation = df.iloc[52258:, :]
train.shape, validation.shape

((52258, 2), (10000, 2))

In [8]:
batch_size = 32
target_vector_size = df['labels'].unique().shape

In [9]:
def train_generator():
    while True:
        for start in range(0, len(train), batch_size):
            x_batch = []
            y_batch = []
            end = min(start + batch_size, len(train))
            train_batch = train.iloc[start:end, :]
            for img_path, label in zip(train_batch['images'], train_batch['labels']):
                img = image.load_img(img_path, target_size=(224, 224))
                x = image.img_to_array(img)
                x = preprocess_input(x)
                
                y = np.zeros(15)
                y[label] = 1
                
                x_batch.append(x)
                y_batch.append(y)
            x_batch = np.array(x_batch, np.float32)
            y_batch = np.array(y_batch, np.float32)
            yield x_batch, y_batch
            
def valid_generator():
    while True:
        for start in range(0, len(validation), batch_size):
            x_batch = []
            y_batch = []
            end = min(start + batch_size, len(validation))
            validation_batch = validation.iloc[start:end, :]
            for img_path, label in zip(validation_batch['images'], validation_batch['labels']):
                img = image.load_img(img_path, target_size=(224, 224))
                x = image.img_to_array(img)
                x = preprocess_input(x)
                
                y = np.zeros(15)
                y[label] = 1
                
                x_batch.append(x)
                y_batch.append(y)
            x_batch = np.array(x_batch, np.float32)
            y_batch = np.array(y_batch, np.float32)
            yield x_batch, y_batch

In [10]:
import tensorflow as tf

def f1_score(y_true, y_pred):
    y_true = tf.cast(y_true, "int32")
    y_pred = tf.cast(tf.round(y_pred), "int32") # implicit 0.5 threshold via tf.round
    y_correct = y_true * y_pred
    sum_true = tf.reduce_sum(y_true, axis=1)
    sum_pred = tf.reduce_sum(y_pred, axis=1)
    sum_correct = tf.reduce_sum(y_correct, axis=1)
    precision = sum_correct / sum_pred
    recall = sum_correct / sum_true
    f_score = 2 * precision * recall / (precision + recall)
    f_score = tf.where(tf.is_nan(f_score), tf.zeros_like(f_score), f_score)
    return tf.reduce_mean(f_score)

/home/winter/miniconda3/envs/dl/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/winter/miniconda3/envs/dl/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [11]:
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization, Dense, Dropout
from keras.models import Model, Sequential
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras import optimizers


model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

top_model = Sequential()
top_model.add(GlobalAveragePooling2D(input_shape=model.output_shape[1:]))
top_model.add(Dense(4096, activation='sigmoid'))
top_model.add(Dropout(0.2))
top_model.add(Dense(2048, activation='sigmoid'))
top_model.add(Dropout(0.5))
top_model.add(Dense(15, activation='softmax'))

top_model.load_weights('best_weights_4096_0.2_2048_0.5_sigmoid.hdf5')

ft_model = Sequential()
for layer in model.layers:
    ft_model.add(layer)
for layer in top_model.layers:
    ft_model.add(layer)

for layer in ft_model.layers[:20]:
    layer.trainable = False
    
ft_model.summary()

callbacks = [EarlyStopping(monitor='val_loss',
                           patience=8,
                           verbose=1,
                           min_delta=1e-4,
                           mode='min'),
             ReduceLROnPlateau(monitor='val_loss',
                               factor=0.1,
                               patience=3,
                               verbose=1,
                               epsilon=1e-4,
                               mode='min'),
             ModelCheckpoint(monitor='val_loss',
                             filepath='best_weights.hdf5',
                             save_best_only=True,
                             save_weights_only=False,
                             mode='min')]

ft_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy', f1_score])

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [13]:
ft_model.fit_generator(generator=train_generator(),
                    steps_per_epoch=np.ceil(float(len(train)) / float(batch_size)),
                    epochs=30,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=valid_generator(),
                    validation_steps=np.ceil(float(len(validation)) / float(batch_size)))

Epoch 1/30
1634/1634 [==============================] - 413s 253ms/step - loss: 0.8091 - acc: 0.7349 - f1_score: 0.6466 - val_loss: 0.8205 - val_acc: 0.7312 - val_f1_score: 0.6384
Epoch 2/30
1634/1634 [==============================] - 433s 265ms/step - loss: 0.7839 - acc: 0.7424 - f1_score: 0.6560 - val_loss: 0.8152 - val_acc: 0.7305 - val_f1_score: 0.6440
Epoch 3/30
1634/1634 [==============================] - 446s 273ms/step - loss: 0.7586 - acc: 0.7522 - f1_score: 0.6641 - val_loss: 0.8140 - val_acc: 0.7326 - val_f1_score: 0.6422
Epoch 4/30
1634/1634 [==============================] - 456s 279ms/step - loss: 0.7376 - acc: 0.7583 - f1_score: 0.6772 - val_loss: 0.8137 - val_acc: 0.7336 - val_f1_score: 0.6468
Epoch 5/30
1634/1634 [==============================] - 463s 283ms/step - loss: 0.7145 - acc: 0.7671 - f1_score: 0.6862 - val_loss: 0.8172 - val_acc: 0.7333 - val_f1_score: 0.6484
Epoch 6/30
1634/1634 [==============================] - 465s 285ms/step - loss: 0.6946 - acc: 0.7734

KeyboardInterrupt: 

In [14]:
test_images = os.listdir('input/dataset/test/')

In [15]:
os.path.basename(test_images[5])

'n04489695_9899_0.jpg'

In [16]:
from tqdm import tqdm
image_name, category = list(), list()
for img_path in tqdm(test_images):
    img_path = os.path.join('input/dataset/test/', img_path)
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = preprocess_input(x)
    image_name.append(os.path.basename(img_path))
    label = np.argmax(ft_model.predict(np.expand_dims(x, axis=0)))
    category.append(label)

100%|██████████| 21273/21273 [04:52<00:00, 72.63it/s]


In [17]:
len(image_name), len(category)

(21273, 21273)

In [18]:
len(os.listdir('input/dataset/test/'))

21273

In [19]:
res = pd.DataFrame({'image_name': image_name, 'category': category}, columns=['image_name', 'category'])
# res = res.append(res.iloc[:5412, :])
print(res.shape)
res.to_csv('submission.csv', index=False)

(21273, 2)


In [20]:
res.category.value_counts()

4     3382
2     2665
3     2654
9     2055
1     2001
6     1712
12    1629
10    1621
8     1230
13    1018
11     408
7      362
5      201
0      170
14     165
Name: category, dtype: int64

In [21]:
preds = ft_model.predict_generator(generator=valid_generator(), 
                               steps=np.ceil(float(len(validation)) / float(batch_size)))
preds = np.argmax(preds, axis=1)
preds.shape

(10000,)

In [22]:
pd.DataFrame({'preds': preds})['preds'].value_counts()

4     1525
2     1286
3     1241
9      989
1      955
6      838
12     764
10     763
8      542
13     468
11     195
7      163
0       97
5       95
14      79
Name: preds, dtype: int64